In [ ]:
!pip install librosa==0.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.2
    Uninstalling librosa-0.10.2:
      Successfully uninstalled librosa-0.10.2


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import tensorflow as tf
from tqdm import tqdm
from glob import glob
from google.colab import drive

import librosa
import librosa.display as dsp
import IPython.display as ipd

warnings.filterwarnings(action='ignore')
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/deep-voice/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/deep-voice/data


In [ ]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #GPU 할당

In [ ]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(813)

In [ ]:
# 저장된 데이터 불러오기
X_train_mel = np.load('X_train_mel.npy')
X_test_mel = np.load('X_test_mel.npy')
X_val_mel = np.load('X_val_mel.npy')

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
y_val = np.load('y_val.npy')

## VGG19

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda, Flatten, Dropout, Dense
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2  # L1, L2 정규화를 위한 라이브러리

input_shape = X_train_mel.shape[1:]  # X_train_mel은 사용자의 데이터로 가정

# 데이터 전처리를 동적으로 처리하는 Lambda 레이어 함수
def dynamic_preprocess(x):
    # 이미지 크기를 (128, 512)으로 조정
    x = tf.image.resize(x, [128, 256])
    return x

vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=(128, 256, 3))
# VGG19의 모든 계층을 동결합니다.
for layer in vgg19.layers:
    layer.trainable = False

# Sequential 모델 구성
model = Sequential([
    Lambda(dynamic_preprocess, input_shape=(128, 256, 3)),  # 입력 데이터 차원 및 크기 조정
    vgg19,
    Flatten(),
    Dropout(0.5),  # 드롭아웃을 적용하여 일부 뉴런을 임의로 비활성화
    # L1, L2 정규화를 적용한 Dense 레이어
    Dense(512, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
    Dense(1, activation='sigmoid')  # 최종 출력 계층을 이진 분류에 맞게 조정
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()

80134624/80134624 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 128, 256, 3)       0         
                                                                 
 vgg19 (Functional)          (None, 4, 8, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 16384)             0         
                                                                 
 dropout (Dropout)           (None, 16384)             0         
                                                                 
 dense (Dense)               (None, 512)               8389120   
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                         

In [ ]:
import numpy as np

def resize_and_duplicate_channel(data):
    # 데이터의 앞쪽 128 프레임만 사용하고, 마지막 차원에 대해 3회 복제
    resized_data = data[:, :, :256, np.newaxis]
    duplicated_data = np.repeat(resized_data, 3, axis=3)
    return duplicated_data

In [ ]:
X_train_mel_resized = resize_and_duplicate_channel(X_train_mel)
X_val_mel_resized = resize_and_duplicate_channel(X_val_mel)
X_test_mel_resized = resize_and_duplicate_channel(X_test_mel)

In [ ]:
X_train_mel_resized.shape

(960, 128, 256, 3)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# ModelCheckpoint 설정
model_checkpoint = ModelCheckpoint(
    'best_model_vgg.h5', # 모델을 저장할 파일 이름
    monitor='val_accuracy', # 모니터링할 대상 ('val_loss'로 설정할 수도 있음)
    mode='max', # 'max'는 정확도를 모니터링할 때, 'min'은 손실을 모니터링할 때 사용
    save_best_only=True # True로 설정하면, 가장 좋은 성능을 보인 모델만 저장
)

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# 모델 학습
history = model.fit(
    X_train_mel_resized,
    y_train,
    epochs=10,
    validation_data=(X_val_mel_resized, y_val),
    callbacks=[early_stopping, model_checkpoint] # 콜백 추가
)

Epoch 1/10
30/30 [==============================] - 682s 23s/step - loss: 695.0236 - accuracy: 0.9458 - val_loss: 585.0825 - val_accuracy: 1.0000
Epoch 2/10
30/30 [==============================] - 709s 24s/step - loss: 492.0841 - accuracy: 0.9979 - val_loss: 398.5992 - val_accuracy: 1.0000
Epoch 3/10
30/30 [==============================] - 710s 24s/step - loss: 322.7713 - accuracy: 0.9979 - val_loss: 248.0730 - val_accuracy: 1.0000
Epoch 4/10
30/30 [==============================] - 682s 23s/step - loss: 190.1919 - accuracy: 0.9990 - val_loss: 134.7675 - val_accuracy: 1.0000
Epoch 5/10
30/30 [==============================] - 679s 23s/step - loss: 95.0309 - accuracy: 0.9990 - val_loss: 58.9810 - val_accuracy: 1.0000
Epoch 6/10
30/30 [==============================] - 671s 23s/step - loss: 37.3297 - accuracy: 0.9990 - val_loss: 20.6104 - val_accuracy: 1.0000
Epoch 7/10
30/30 [==============================] - 678s 23s/step - loss: 15.0767 - accuracy: 0.9990 - val_loss: 11.1116 - val_a

In [ ]:
# 모델 불러오기
from tensorflow.keras.models import load_model
best_model = load_model('best_model_vgg.h5')

# 불러온 모델로 테스트 데이터셋 성능 평가
test_loss, test_acc = best_model.evaluate(X_test_mel_resized, y_test)
print(f'\nTest accuracy: {test_acc}, Test loss: {test_loss}')

10/10 [==============================] - 166s 17s/step - loss: 585.0825 - accuracy: 1.0000

Test accuracy: 1.0, Test loss: 585.0824584960938


In [ ]:
from tensorflow.keras.models import load_model

def audio_to_mel_spectrogram(audio_path):
    # 오디오 파일 로드
    y, sr = librosa.load(audio_path)
    S = librosa.feature.melspectrogram(y, sr=16000, n_mels=128)
    S_DB = librosa.power_to_db(S, ref=np.max)
    return S_DB

def resize_and_duplicate_channel(data):
    # 데이터의 크기 조정 및 채널 복제를 위한 수정
    # 모델의 입력 형태에 맞추기 위해 128 x 256 크기로 조정
    if data.shape[1] < 256:
        # 데이터가 256보다 작은 경우, 부족한 부분을 0으로 채움
        data = np.pad(data, ((0, 0), (0, 256 - data.shape[1])), 'constant', constant_values=(0))
    elif data.shape[1] > 256:
        # 데이터가 256보다 큰 경우, 처음 256개만 사용
        data = data[:, :256]

    # 마지막 차원에 대해 3회 복제하여 채널 차원 추가
    data = np.expand_dims(data, axis=-1)
    duplicated_data = np.repeat(data, 3, axis=2)
    return duplicated_data

# 모델 로드
best_model = load_model('best_model_vgg.h5')

# 오디오 파일 경로
audio_path = '/content/2-ko-100018_1_1-M-27-100018_1_1_1.wav'

# 오디오 파일을 Mel-spectrogram으로 변환
mel_spectrogram = audio_to_mel_spectrogram(audio_path)

# 데이터 전처리
preprocessed_data = resize_and_duplicate_channel(mel_spectrogram)
preprocessed_data = np.expand_dims(preprocessed_data, axis=0)  # 배치 차원 추가

# 모델을 사용하여 예측
prediction = best_model.predict(preprocessed_data)

# 예측 결과 출력
if prediction[0] > 0.5:
    print("딥보이스입니다.")
else:
    print("딥보이스가 아닙니다.")

1/1 [==============================] - 2s 2s/step
딥보이스입니다.


## GoogleNet

In [ ]:
import torch
from torchvision import transforms, models
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToPILImage
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor

class CustomMelSpectrogramDataset(Dataset):
    def __init__(self, X_data, y_data, transform=None):
        self.X_data = X_data
        self.y_data = y_data
        self.transform = transform

    def __getitem__(self, idx):
        mel_spec = self.X_data[idx]

        # 이미 데이터가 (Height, Width) 형태라고 가정할 때,
        # 여기서는 데이터가 torch.Tensor 형태임을 확인
        if not isinstance(mel_spec, torch.Tensor):
            raise ValueError(f"Expected torch.Tensor, got {type(mel_spec)}.")

        # (높이, 너비, 채널)에서 (채널, 높이, 너비)로 차원 재배열
        mel_spec = mel_spec.permute(2, 0, 1)

        # 멜 스펙트로그램을 PIL 이미지로 변환 시도
        try:
            mel_spec_image = ToPILImage()(mel_spec)
        except ValueError as e:
            # 에러 핸들링
            print(f"Error converting mel spectrogram to image: {e}")
            # 에러 발생 시 특정 처리 로직 (예: 기본 이미지 할당)
            mel_spec_image = None  # or some default handling

        # 추가 변형이 있으면 적용
        if self.transform:
            mel_spec_image = self.transform(mel_spec_image)

        label = self.y_data[idx]
        return mel_spec_image, label

    def __len__(self):
        # 데이터셋 내 아이템의 총 개수 반환
        return len(self.X_data)

# 멜스펙트로그램 데이터를 텐서로 변환
X_train_mel_tensor = torch.tensor(X_train_mel, dtype=torch.float32)
X_val_mel_tensor = torch.tensor(X_val_mel, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# 입력 데이터의 차원 변환 및 채널 조정을 위한 전처리 파이프라인
# 데이터의 크기를 (224, 224)로 조정하고, 1채널을 3채널로 복제
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.Grayscale(num_output_channels=3), # 1채널을 3채널로 변경
    transforms.ToTensor()
])

# 데이터셋 생성
train_dataset = CustomMelSpectrogramDataset(X_train_mel_tensor, y_train_tensor, transform=transform)
val_dataset = CustomMelSpectrogramDataset(X_val_mel_tensor, y_val_tensor, transform=transform)

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# GoogLeNet 모델 로드
model = models.googlenet(pretrained=True)

# 마지막 FC 레이어를 사용자 정의 데이터셋에 맞게 변경
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# GPU 사용 가능 여부 확인 후 모델을 GPU로 전송
model.to(device)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 78.2MB/s]


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# DataLoader에서 첫 번째 배치의 형태를 출력합니다.
for batch_idx, (data, target) in enumerate(train_loader):
    print(f'배치 인덱스: {batch_idx}')
    print(f'데이터 형태: {data.shape}')  # 예: torch.Size([32, 10])
    print(f'타겟 형태: {target.shape}')  # 예: torch.Size([32])
    break  # 첫 번째 배치만 출력되도록 합니다.

배치 인덱스: 0
데이터 형태: torch.Size([32, 3, 224, 224])
타겟 형태: torch.Size([32])


In [ ]:
import numpy as np

class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 종료"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Arguments:
            patience (int): 개선된 validation loss가 관찰되지 않는 epoch 횟수 후에 학습을 중단할 횟수
            verbose (bool): 조기 종료 메시지를 출력할지 결정
            delta (float): 개선으로 간주되기 위한 최소 변화량
            path (str): 모델 저장 경로
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            if val_loss < self.best_loss:
                self.save_checkpoint(val_loss, model)
                self.best_loss = val_loss
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장합니다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# EarlyStopping 객체 초기화
early_stopping = EarlyStopping(patience=20, verbose=True)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('[Epoch %d] loss: %.3f' % (epoch + 1, running_loss))

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print('[Epoch %d] val_loss: %.3f, Accuracy: %.2f %%' % (epoch + 1, val_loss, val_accuracy))

    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("조기 종료")
        break

[Epoch 1] loss: 31.059
[Epoch 1] val_loss: 3.336, Accuracy: 96.88 %
[Epoch 2] loss: 4.425
[Epoch 2] val_loss: 0.479, Accuracy: 99.38 %
Validation loss decreased (inf --> 0.479194).  Saving model ...
[Epoch 3] loss: 1.880
[Epoch 3] val_loss: 0.322, Accuracy: 99.38 %
Validation loss decreased (0.479194 --> 0.321925).  Saving model ...
[Epoch 4] loss: 1.297
[Epoch 4] val_loss: 0.203, Accuracy: 99.69 %
Validation loss decreased (0.321925 --> 0.203315).  Saving model ...
[Epoch 5] loss: 0.771
[Epoch 5] val_loss: 0.161, Accuracy: 99.69 %
Validation loss decreased (0.203315 --> 0.161418).  Saving model ...
[Epoch 6] loss: 0.941
[Epoch 6] val_loss: 0.134, Accuracy: 100.00 %
Validation loss decreased (0.161418 --> 0.133798).  Saving model ...
[Epoch 7] loss: 0.553
[Epoch 7] val_loss: 0.122, Accuracy: 100.00 %
Validation loss decreased (0.133798 --> 0.122250).  Saving model ...
[Epoch 8] loss: 0.399
[Epoch 8] val_loss: 0.110, Accuracy: 99.69 %
Validation loss decreased (0.122250 --> 0.109535).  

## ResNet18

In [ ]:
import torch
from torchvision import transforms, models
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToPILImage
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor

class CustomMelSpectrogramDataset(Dataset):
    def __init__(self, X_data, y_data, transform=None):
        self.X_data = X_data
        self.y_data = y_data
        self.transform = transform

    def __getitem__(self, idx):
        mel_spec = self.X_data[idx]

        # 이미 데이터가 (Height, Width) 형태라고 가정할 때,
        # 여기서는 데이터가 torch.Tensor 형태임을 확인
        if not isinstance(mel_spec, torch.Tensor):
            raise ValueError(f"Expected torch.Tensor, got {type(mel_spec)}.")

        # (높이, 너비, 채널)에서 (채널, 높이, 너비)로 차원 재배열
        mel_spec = mel_spec.permute(2, 0, 1)

        # 멜 스펙트로그램을 PIL 이미지로 변환 시도
        try:
            mel_spec_image = ToPILImage()(mel_spec)
        except ValueError as e:
            # 에러 핸들링
            print(f"Error converting mel spectrogram to image: {e}")
            # 에러 발생 시 특정 처리 로직 (예: 기본 이미지 할당)
            mel_spec_image = None  # or some default handling

        # 추가 변형이 있으면 적용
        if self.transform:
            mel_spec_image = self.transform(mel_spec_image)

        label = self.y_data[idx]
        return mel_spec_image, label

    def __len__(self):
        # 데이터셋 내 아이템의 총 개수 반환
        return len(self.X_data)

# 멜스펙트로그램 데이터를 텐서로 변환
X_train_mel_tensor = torch.tensor(X_train_mel, dtype=torch.float32)
X_val_mel_tensor = torch.tensor(X_val_mel, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# 입력 데이터의 차원 변환 및 채널 조정을 위한 전처리 파이프라인
# 데이터의 크기를 (224, 224)로 조정하고, 1채널을 3채널로 복제
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.Grayscale(num_output_channels=3), # 1채널을 3채널로 변경
    transforms.ToTensor()
])

# 데이터셋 생성
train_dataset = CustomMelSpectrogramDataset(X_train_mel_tensor, y_train_tensor, transform=transform)
val_dataset = CustomMelSpectrogramDataset(X_val_mel_tensor, y_val_tensor, transform=transform)

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# ResNet 모델 로드
model = models.resnet18(pretrained=True)  # ResNet18 모델을 로드

# 마지막 FC 레이어를 사용자 정의 데이터셋에 맞게 변경
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # 클래스 개수를 맞추기 위해 10으로 설정

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# GPU 사용 가능 여부 확인 후 모델을 GPU로 전송
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# DataLoader에서 첫 번째 배치의 형태를 출력합니다.
for batch_idx, (data, target) in enumerate(train_loader):
    print(f'배치 인덱스: {batch_idx}')
    print(f'데이터 형태: {data.shape}')  # 예: torch.Size([32, 10])
    print(f'타겟 형태: {target.shape}')  # 예: torch.Size([32])
    break  # 첫 번째 배치만 출력되도록 합니다.

배치 인덱스: 0
데이터 형태: torch.Size([32, 3, 224, 224])
타겟 형태: torch.Size([32])


In [ ]:
import numpy as np

class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 종료"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Arguments:
            patience (int): 개선된 validation loss가 관찰되지 않는 epoch 횟수 후에 학습을 중단할 횟수
            verbose (bool): 조기 종료 메시지를 출력할지 결정
            delta (float): 개선으로 간주되기 위한 최소 변화량
            path (str): 모델 저장 경로
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            if val_loss < self.best_loss:
                self.save_checkpoint(val_loss, model)
                self.best_loss = val_loss
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장합니다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# EarlyStopping 객체 초기화
early_stopping = EarlyStopping(patience=20, verbose=True)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('[Epoch %d] loss: %.3f' % (epoch + 1, running_loss))

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print('[Epoch %d] val_loss: %.3f, Accuracy: %.2f %%' % (epoch + 1, val_loss, val_accuracy))

    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("조기 종료")
        break

[Epoch 1] loss: 13.456
[Epoch 1] val_loss: 1.484, Accuracy: 96.25 %
[Epoch 2] loss: 1.031
[Epoch 2] val_loss: 0.304, Accuracy: 99.38 %


KeyboardInterrupt: 